# Demo 12: Error Pattern Visualizer

This notebook demonstrates the **Error Pattern Visualizer** for automatically discovering correction patterns in error signals.

## Why This Tool Exists

During optimization of `fast_zetas.py`, we manually:
- Computed error = actual - predicted
- Analyzed error using FFT to find dominant frequencies
- Fitted sinusoidal corrections
- Applied recursive refinement to residuals

This tool **automates all of that**. It discovers patterns you might not think of and generates production-ready correction code.

## Key Insight

**Errors are not random—they contain structure that reveals missing terms.**

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from error_pattern_visualizer import (
    ErrorPatternAnalyzer,
    ErrorVisualizer
)

print("✓ Imports successful")

## 1. Basic Error Analysis

Analyze a simple error pattern: sin wave with missing linear trend.

In [ ]:
# Create synthetic data
x = np.linspace(0, 10, 100)
actual = np.sin(x) + 0.1 * x  # Sin + linear trend
predicted = np.sin(x)  # Missing linear term

# Analyze errors
analyzer = ErrorPatternAnalyzer(actual, predicted, x, name="Sin + Linear")
report = analyzer.analyze_all()

# Print summary
report.print_summary()

In [ ]:
# Show generated correction code
print("\nTop correction code:")
print("=" * 50)
print(report.suggestions[0].code_snippet)

## 2. Spectral Pattern Detection

Detect periodic patterns using FFT analysis.

In [ ]:
# Create data with multiple harmonics
x = np.linspace(0, 10, 200)
actual = (np.sin(2 * np.pi * 1.0 * x) + 
          0.5 * np.sin(2 * np.pi * 3.0 * x) + 
          0.3 * np.sin(2 * np.pi * 5.0 * x))
predicted = np.sin(2 * np.pi * 1.0 * x)  # Missing harmonics

analyzer = ErrorPatternAnalyzer(actual, predicted, x, name="Multiple Harmonics")

# Analyze spectral patterns
spectral = analyzer.analyze_spectral(n_harmonics=5)
print(f"Detected {len(spectral.frequencies)} harmonics")
print(f"Explained variance: {spectral.explained_variance:.1%}")
print(f"\nDominant frequencies:")
for i, (freq, amp) in enumerate(zip(spectral.frequencies[:5], spectral.amplitudes[:5]), 1):
    print(f"  {i}. Frequency: {freq:.2f}, Amplitude: {amp:.4f}")

In [ ]:
# Visualize spectral analysis
visualizer = ErrorVisualizer(analyzer)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
visualizer.plot_error_signal(ax=ax1)
visualizer.plot_spectral_analysis(ax=ax2)
plt.tight_layout()
plt.show()

## 3. Polynomial Trend Detection

Detect systematic bias using polynomial fitting.

In [ ]:
# Create data with polynomial error
x = np.linspace(0, 10, 100)
actual = x**2 + 2*x + 5
predicted = x**2  # Missing linear and constant terms

analyzer = ErrorPatternAnalyzer(actual, predicted, x, name="Polynomial Error")
poly = analyzer.analyze_polynomial(max_degree=5)

print(f"Detected polynomial degree: {poly.degree}")
print(f"R²: {poly.r_squared:.6f}")
print(f"Coefficients: {poly.coefficients}")
print(f"\nGenerated code:")
print(poly.to_correction_code())

## 4. Apply Corrections

Apply discovered corrections and measure improvement.

In [ ]:
# Create data with known error
x = np.linspace(1, 10, 100)
actual = np.log(x) + 5
predicted = np.log(x)  # Missing constant

analyzer = ErrorPatternAnalyzer(actual, predicted, x, name="Log + Constant")

print(f"Original RMSE: {analyzer.rmse:.6f}")

# Get suggestions and apply best
suggestions = analyzer.suggest_corrections(top_k=3)
best = suggestions[0]

print(f"\nApplying: {best.description}")
corrected = analyzer.apply_correction(best)

print(f"Corrected RMSE: {corrected.rmse:.6f}")
improvement = (1 - corrected.rmse / analyzer.rmse) * 100
print(f"Improvement: {improvement:.1f}%")

## 5. Recursive Refinement

Automatically apply corrections until convergence.

In [ ]:
# Create data with multiple error patterns
x = np.linspace(0, 10, 150)
actual = (np.sin(x) + 
          0.05 * x**2 + 
          0.2 * np.sin(2 * np.pi * 2.0 * x) +
          0.1 * x)
predicted = np.sin(x)  # Missing polynomial, harmonic, and linear

analyzer = ErrorPatternAnalyzer(actual, predicted, x, name="Complex Error")

# Apply recursive refinement
history = analyzer.recursive_refinement(
    max_depth=5, 
    improvement_threshold=0.01  # Stop if improvement < 1%
)

print(f"Initial RMSE: {history.initial_rmse:.6f}")
print(f"Final RMSE:   {history.final_rmse:.6f}")
print(f"Improvement:  {history.improvement:.1%}")
print(f"Depth:        {history.depth} corrections\n")

print("Applied corrections:")
for i, corr in enumerate(history.corrections_applied, 1):
    print(f"  {i}. {corr.description}")
    print(f"     RMSE after: {history.rmse_history[i]:.6f}")

In [ ]:
# Plot convergence
history.plot_convergence()

## 6. Scale-Dependent Errors

Detect errors that depend on the magnitude of x.

In [ ]:
# Create data with scale-dependent error
x = np.linspace(1, 100, 200)
actual = np.log(x) + 0.1 * np.sqrt(x)
predicted = np.log(x)  # Missing sqrt term

analyzer = ErrorPatternAnalyzer(actual, predicted, x, name="Scale-Dependent")
scale = analyzer.analyze_scale_dependence(n_bins=10)

if scale.scale_function:
    model = scale.scale_params.get('model', 'unknown')
    r2 = scale.scale_params.get('r2', 0.0)
    print(f"Detected scale pattern: {model}")
    print(f"R²: {r2:.4f}")
    print(f"\nGenerated code:")
    print(scale.to_correction_code())
else:
    print("No significant scale dependence detected")

In [ ]:
# Visualize scale dependence
visualizer = ErrorVisualizer(analyzer)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
visualizer.plot_error_signal(ax=ax1)
visualizer.plot_scale_dependence(ax=ax2)
plt.tight_layout()
plt.show()

## 7. Autocorrelation Detection

Detect recursive patterns in errors.

In [ ]:
# Create data with autocorrelated error
np.random.seed(42)
x = np.arange(200)
# Create AR(1) error: e[t] = 0.8*e[t-1] + noise
error_ar = np.zeros(200)
for i in range(1, 200):
    error_ar[i] = 0.8 * error_ar[i-1] + np.random.randn() * 0.1

actual = x * 0.5 + error_ar
predicted = x * 0.5

analyzer = ErrorPatternAnalyzer(actual, predicted, x, name="Autocorrelated Error")
autocorr = analyzer.analyze_autocorrelation(max_lag=50)

print(f"Significant lags: {autocorr.significant_lags[:5]}")
if autocorr.ar_order:
    print(f"Suggested AR order: {autocorr.ar_order}")
    print(f"AR coefficients: {autocorr.ar_coefficients}")

In [ ]:
# Visualize autocorrelation
visualizer = ErrorVisualizer(analyzer)
visualizer.plot_autocorrelation()

## 8. Full Dashboard

Comprehensive visualization of all patterns.

In [ ]:
# Create complex error for dashboard
x = np.linspace(0, 10, 200)
actual = (np.sin(x) + 
          0.1 * x + 
          0.3 * np.sin(2 * np.pi * 3.0 * x))
predicted = np.sin(x)

analyzer = ErrorPatternAnalyzer(actual, predicted, x, name="Dashboard Example")
visualizer = ErrorVisualizer(analyzer)

# Show full dashboard
visualizer.plot_full_dashboard(figsize=(16, 12))

## 9. Real-World Example: Zeta Zero Approximation

Analyze errors in a zeta zero initial guess formula.

In [ ]:
# Simulate zeta zero approximation error
# (In practice, you'd use actual zeta zeros and your formula)
n = np.arange(1, 101)

# Simplified: actual ~ 2π(n-11/8)/log(n) + harmonics
actual_zeros = 2 * np.pi * (n - 11/8) / np.log(n + 1e-10)

# Initial guess without harmonics
predicted_zeros = 2 * np.pi * (n - 11/8) / np.log(n + 1e-10) * 0.95  # Simplified

analyzer = ErrorPatternAnalyzer(
    actual_zeros, 
    predicted_zeros, 
    n, 
    name="Zeta Zero Approximation"
)

report = analyzer.analyze_all(max_corrections=5)
report.print_summary()

print("\nTop 3 correction codes:")
for i, sug in enumerate(report.suggestions[:3], 1):
    print(f"\n{i}. {sug.description}")
    print("-" * 50)
    print(sug.code_snippet)

## 10. Export Corrections as Production Code

Generate a complete correction function.

In [ ]:
def generate_correction_function(analyzer, suggestions, func_name="improved_formula"):
    """Generate a complete Python function with all corrections."""
    code = [
        f"def {func_name}(x, original_prediction):",
        "    import numpy as np",
        "    prediction = original_prediction.copy()",
        "    error = np.zeros_like(x)  # For AR models",
        ""
    ]
    
    for i, suggestion in enumerate(suggestions, 1):
        code.append(f"    # Correction {i}: {suggestion.description}")
        # Indent the correction code
        correction_lines = suggestion.code_snippet.split('\n')
        for line in correction_lines:
            if line.strip():
                code.append(f"    {line}")
        code.append("")
    
    code.append("    return prediction")
    return "\n".join(code)

# Generate function
x = np.linspace(0, 10, 100)
actual = np.sin(x) + 0.1 * x
predicted = np.sin(x)

analyzer = ErrorPatternAnalyzer(actual, predicted, x)
suggestions = analyzer.suggest_corrections(top_k=2)

correction_code = generate_correction_function(analyzer, suggestions)
print("Generated production code:")
print("=" * 70)
print(correction_code)

## Summary

The Error Pattern Visualizer provides:

1. **Automatic Pattern Discovery** - Finds patterns you might miss
2. **Spectral Analysis** - Detects periodic corrections via FFT
3. **Polynomial Trends** - Identifies systematic bias
4. **Autocorrelation** - Discovers recursive patterns
5. **Scale Dependence** - Detects x-dependent errors
6. **Code Generation** - Produces executable correction code
7. **Recursive Refinement** - Applies corrections until convergence
8. **Visualization** - Comprehensive error analysis dashboards

### When to Use

- Optimizing mathematical formulas (like zeta zeros)
- Improving signal approximations
- Analyzing model prediction errors
- Discovering missing correction terms
- Any scenario with actual vs predicted values

### Key Insight

**Errors contain structure. This tool extracts that structure and converts it into corrections.**